In [1]:
import os
# os.environ['SCOPE'] = 'uat'
# os.environ['CUSTOMER'] = 'regeneron'
# os.environ['NameSpace'] = 'airflow'
if os.environ['CUSTOMER'] == 'CSIO':
    %run /home/jovyan/work/DAGS_Airflow/icm_config_csio.ipynb
elif os.environ['CUSTOMER'] == 'globelife':
    %run /home/jovyan/work/DAGS_Airflow/icm_config.ipynb
elif os.environ['CUSTOMER'] == 'regeneron':
    %run /home/jovyan/work/DAGS_Airflow/icm_config_regeneron.ipynb

In [2]:
%run /home/jovyan/work/DAGS_Airflow/sparkConf.ipynb

In [3]:
Activity = os.environ['CUSTOMER']
Activity

'regeneron'

In [4]:
import os
# Set executor python spark environments
os.environ['PYSPARK_PYTHON'] = "/miniconda/bin/python"
from utils.spark_util import KubeflowSparkUtil
sparkConf = KubeflowSparkUtil.get_spark_conf(KubeflowSparkUtil, spark_conf, 'AiopsPreprocess-Airflow-'+Activity, 'k8s://https://kubernetes.default.svc.cluster.local:443')
from pyspark.sql import SQLContext
spark=  KubeflowSparkUtil(sparkConf).get_spark_session()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [47]:
from utils.aiops_config import AiopsConfig
from core.icm.icm_preprocessor import ICMPreprocessor
from core.icm.incident_processor import IncidentProcessor
from core.icm.IcmAutomation import IcmAutomation
from core.icm.baseline_summary import BaselineSummary
from core.icm.cost_baselining import ICMComputeBaseline, ICMStorageBaseline

In [48]:
customer = os.environ['CUSTOMER']
aiopsconfig = AiopsConfig(customer,2, icm_configurations,icm_modelDBconfig)

In [50]:
preprocessor = ICMPreprocessor(aiopsconfig,spark)

In [52]:
compute_preprocessing_df, storage_preprocessing_df = preprocessor.preprocess()

compute pre process start
storage pre process start
end


In [53]:
# MAGIC %md ##Incident Data Processing

# COMMAND ----------

incident_processor = IncidentProcessor(aiopsconfig,spark)
incident_processor.process_incidents()

DataFrame[Incident Number: string, Urgency: string, Impact: string, Priority: string, Reported Source: string, Status: string, Product Categorization Tier 1: string, Operational Categorization Tier 1: string, Operational Categorization Tier 2: string, Detailed Description: string, Summary: string, Resolution: string, Assignee Support Group: string, Assignee Support Organization: string, CI Name: string, Reported Date Time (Company Offset): timestamp, Last Resolved Date Time (Company Offset): timestamp, Closed Date Time (Company Offset): timestamp, Last Modified Date Time (Company Offset): timestamp, Re-Opened Date Time (Company Offset): timestamp, customer_id: int]

In [54]:
# MAGIC %md ##Monitoring

# COMMAND ----------

preprocessor.analyze_data_quality(compute_preprocessing_df,storage_preprocessing_df)

In [55]:
# MAGIC %md ##Cost Baseline

# COMMAND ----------

compute_baseline = ICMComputeBaseline(aiopsconfig,spark)
compute_baseline.execute_baseline()

r2
+------+----------+------------+------------+--------+-----+--------------+---------------+------------------+-------------+---------+---------+-------------------+-----------------------+---------------+------------+-----------+----------+----------+-----------------+--------------+-----------------+-----------------+-----------+
|DEVICE|CHECK_TIME|BL_CPU_ALLOC|BL_MEM_ALLOC|LOCATION|TOWER|RECOMMENDATION|BL_CPU_MEM_COST|BL_CPU_MEM_SAVINGS|BASELINE_DATE|CPU_ALLOC|MEM_ALLOC|ACTUAL_CPU_MEM_COST|REALIZED_CPU_MEM_SAVING|TOTAL_INCIDENTS|action_taken|change_date|change_cpu|change_mem|Baseline_incident|compute_action|INCIDENT_COUNT_P1|INCIDENT_COUNT_P2|customer_id|
+------+----------+------------+------------+--------+-----+--------------+---------------+------------------+-------------+---------+---------+-------------------+-----------------------+---------------+------------+-----------+----------+----------+-----------------+--------------+-----------------+-----------------+-----------

In [56]:
# COMMAND ----------

storage_baseline = ICMStorageBaseline(aiopsconfig,spark)
storage_baseline.execute_baseline()

In [57]:
# MAGIC %md ##Automation and Feedback

# COMMAND ----------

icm_automation = IcmAutomation(aiopsconfig,spark)
icm_automation.automate_recommendation()
icm_automation.process_feedback()

In [ ]:
# MAGIC %md ## Feedback summary and rejection reasons count

# COMMAND ----------

baseline_summary = BaselineSummary(aiopsconfig,spark)
baseline_summary.feedback_summary()
baseline_summary.get_rejection_reasons()

In [24]:
spark.stop()